In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

from imblearn.over_sampling import SMOTE


Tutorial link: https://www.tensorflow.org/tutorials/structured_data/feature_columns

In [3]:
dataframe = pd.read_csv("demographic.csv")

In [4]:
dataframe.dropna(inplace=True)

In [5]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19813 entries, 0 to 19812
Data columns (total 48 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   raceeth  19813 non-null  float64
 1   q1       19813 non-null  float64
 2   q2       19813 non-null  float64
 3   q3       19813 non-null  float64
 4   q4       19813 non-null  float64
 5   q6       19813 non-null  float64
 6   q7       19813 non-null  float64
 7   q64      19813 non-null  float64
 8   q65      19813 non-null  float64
 9   q8       19813 non-null  float64
 10  q9       19813 non-null  float64
 11  q10      19813 non-null  float64
 12  q11      19813 non-null  float64
 13  q12      19813 non-null  float64
 14  q17      19813 non-null  float64
 15  q19      19813 non-null  float64
 16  q26      19813 non-null  float64
 17  q27      19813 non-null  float64
 18  q28      19813 non-null  float64
 19  q29      19813 non-null  float64
 20  q30      19813 non-null  float64
 21  q31      198

In [6]:
dataframe['target'] = dataframe['MB']


In [7]:
dataframe.head()

,raceeth,q1,q2,q3,q4,q6,q7,q64,q65,q8,...,q79,q80,q81,q82,q84,q85,q87,q92,MB,target
0,0.573335,0.333333,0.0,0.00,0.799704,0.464789,0.336362,0.2,0.0,0.75,...,0.00,0.714286,0.0,0.0,0.00,0.666667,0.200836,0.2,0.0,0.0
1,0.571429,0.500000,1.0,0.00,1.000000,0.530048,0.249643,0.0,0.0,1.00,...,0.00,0.428571,0.0,0.0,0.50,0.333333,0.500000,0.0,0.0,0.0
2,0.571429,0.666667,1.0,0.50,1.000000,0.507042,0.290868,0.4,0.0,1.00,...,0.00,1.000000,0.0,0.0,0.25,1.000000,0.333333,0.0,1.0,1.0
3,0.571429,0.833333,0.0,0.25,1.000000,0.530048,0.249643,0.0,0.0,0.75,...,0.00,1.000000,0.0,0.0,0.50,0.500000,0.333333,0.0,1.0,1.0
4,0.571429,0.333333,1.0,0.00,1.000000,0.746479,0.169684,0.0,0.0,1.00,...,0.25,0.714286,0.0,0.0,0.50,0.666667,0.500000,0.0,1.0,1.0


In [8]:
dataframe = dataframe.drop(['MB'], axis=1)

In [9]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.125)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

13868 train examples
1982 validation examples
3963 test examples


In [10]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [11]:
X_train = train.drop('target', axis=1)  
y_train = train['target']  

In [12]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [13]:
train_resampled = X_train_resampled.copy()
train_resampled['target'] = y_train_resampled

In [14]:
feature_columns = []

for header in ["raceeth", "q1", "q2", "q3", "q4", "q6", "q7", "q64", "q65", "q8", "q9", "q10", "q11", "q12", "q17",	"q19", "q26", "q27", "q28",	"q29", "q30", "q31", "q35", "q38", "q39", "q46", "q49", "q50", "q51", "q52", "q53", "q54", "q55",	"q58", "q59", "q60", "q61",	"q74", "q75", "q79", "q80", "q81", "q82", "q84", "q85", "q87", "q92"]:
    feature_columns.append(feature_column.numeric_column(header))


Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


In [15]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [16]:
batch_size = 32
train_ds = df_to_dataset(train_resampled, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [17]:
model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(.3),
    layers.Dense(1)
])

In [18]:
from sklearn.utils.class_weight import compute_class_weight

classes = np.unique(train['target'])
class_weights = compute_class_weight(
    class_weight='balanced', 
    classes=classes, 
    y=train['target']
)
class_weight_dict = {i: class_weights[i] for i in range(len(classes))}
print(f"Class weights: {class_weight_dict}")

Class weights: {0: 1.069236700077101, 1: 0.9391846133008263}


In [19]:
def specificity(y_true, y_pred):
    y_pred = tf.round(tf.nn.sigmoid(y_pred))  
    true_negatives = tf.reduce_sum(tf.cast((y_true == 0) & (y_pred == 0), 'float'))
    false_positives = tf.reduce_sum(tf.cast((y_true == 0) & (y_pred == 1), 'float'))
    
    specificity = true_negatives / (true_negatives + false_positives + tf.keras.backend.epsilon())
    return specificity

In [20]:
def f1_score(y_true, y_pred):
    y_pred = tf.round(tf.nn.sigmoid(y_pred))  
    tp = tf.reduce_sum(tf.cast(y_true * y_pred, 'float'))
    fp = tf.reduce_sum(tf.cast((1 - y_true) * y_pred, 'float'))
    fn = tf.reduce_sum(tf.cast(y_true * (1 - y_pred), 'float'))

    precision = tp / (tp + fp + tf.keras.backend.epsilon())
    recall = tp / (tp + fn + tf.keras.backend.epsilon())
    f1 = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
    return f1

In [21]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[
                  'accuracy',
                  tf.keras.metrics.AUC(name='auc'),
                  specificity,
                  tf.keras.metrics.Recall(name='sensitivity'),
                  f1_score
                      ]
             )

In [22]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [23]:
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6)


In [24]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[early_stopping, lr_scheduler]
)

Epoch 1/50


2025-01-11 13:20:44.402982: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 7031 of 14766
2025-01-11 13:20:54.402971: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 7880 of 14766
2025-01-11 13:21:14.422343: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 10367 of 14766
2025-01-11 13:21:34.393413: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 11023 of 14766
2025-01-11 13:21:44.419715: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 11298 of 14766
2025-01-11 13:21:54.425490: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may 

139/462 [========>.....................] - ETA: 0s - loss: 0.4749 - accuracy: 0.8500 - auc: 0.9010 - specificity: 0.8524 - sensitivity: 0.7458 - f1_score: 0.8526        

2025-01-11 13:23:25.675901: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


462/462 [==============================] - 173s 2ms/step - loss: 0.3364 - accuracy: 0.8965 - auc: 0.9286 - specificity: 0.8965 - sensitivity: 0.8438 - f1_score: 0.8965 - val_loss: 0.2167 - val_accuracy: 0.9485 - val_auc: 0.9686 - val_specificity: 0.8885 - val_sensitivity: 0.9505 - val_f1_score: 0.9430 - lr: 0.0010
Epoch 2/50


...

462/462 [==============================] - 108s 1ms/step - loss: 0.0347 - accuracy: 0.9957 - auc: 0.9989 - specificity: 0.9978 - sensitivity: 0.9917 - f1_score: 0.9978 - val_loss: 0.0365 - val_accuracy: 0.9955 - val_auc: 0.9981 - val_specificity: 0.9937 - val_sensitivity: 0.9927 - val_f1_score: 0.9962 - lr: 1.0000e-06
Epoch 50/50


2025-01-11 15:04:14.193101: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 9365 of 14766
2025-01-11 15:04:34.157200: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 11790 of 14766
2025-01-11 15:04:44.171410: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 12149 of 14766
2025-01-11 15:04:54.178964: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 12504 of 14766
2025-01-11 15:05:14.155186: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 13190 of 14766
2025-01-11 15:05:34.176130: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may

117/462 [======>.......................] - ETA: 0s - loss: 0.0337 - accuracy: 0.9960 - auc: 0.9989 - specificity: 0.9986 - sensitivity: 0.9936 - f1_score: 0.9982    

2025-01-11 15:06:00.300901: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


462/462 [==============================] - 117s 1ms/step - loss: 0.0345 - accuracy: 0.9961 - auc: 0.9989 - specificity: 0.9974 - sensitivity: 0.9928 - f1_score: 0.9976 - val_loss: 0.0365 - val_accuracy: 0.9955 - val_auc: 0.9981 - val_specificity: 0.9937 - val_sensitivity: 0.9927 - val_f1_score: 0.9962 - lr: 1.0000e-06


In [57]:
model.fit(
    test_ds,
    epochs=50,
)

Epoch 1/50
124/124 [==============================] - 0s 1ms/step - loss: 0.0429 - accuracy: 0.9919 - auc: 0.9971 - specificity: 0.9908 - sensitivity: 0.9874 - f1_score: 0.9926
Epoch 2/50
124/124 [==============================] - 0s 1ms/step - loss: 0.0429 - accuracy: 0.9924 - auc: 0.9971 - specificity: 0.9910 - sensitivity: 0.9893 - f1_score: 0.9926
Epoch 3/50
124/124 [==============================] - 0s 1ms/step - loss: 0.0432 - accuracy: 0.9924 - auc: 0.9966 - specificity: 0.9938 - sensitivity: 0.9883 - f1_score: 0.9938
Epoch 4/50
124/124 [==============================] - 0s 1ms/step - loss: 0.0434 - accuracy: 0.9919 - auc: 0.9973 - specificity: 0.9917 - sensitivity: 0.9879 - f1_score: 0.9936
Epoch 5/50
124/124 [==============================] - 0s 1ms/step - loss: 0.0426 - accuracy: 0.9922 - auc: 0.9957 - specificity: 0.9938 - sensitivity: 0.9874 - f1_score: 0.9932
Epoch 6/50
124/124 [==============================] - 0s 1ms/step - loss: 0.0433 - accuracy: 0.9917 - auc: 0.9966 -

In [58]:
weights = model.get_weights()


In [59]:
first_layer_weights = weights[0]  

weight_df = pd.DataFrame(
    first_layer_weights[:8],  
    columns=[f'Hidden_Node_{i}' for i in range(first_layer_weights.shape[1])], 
    index=X_train.columns[:8]  
)

In [60]:
feature_importance = weight_df.abs().sum(axis=1).sort_values(ascending=False)

print(feature_importance)

raceeth    4.103426
q7         3.959653
q64        3.787702
q2         3.423419
q3         3.236468
q6         3.150844
q1         3.137364
q4         2.557290
dtype: float32


In [61]:
weight_mb = pd.DataFrame(first_layer_weights, columns=[f'Hidden_Node_{i}' for i in range(first_layer_weights.shape[1])], index=X_train.columns)

mb_importance = weight_mb.abs().sum(axis=1).sort_values(ascending=False)
print(mb_importance)

q27        6.554233
q74        5.102590
q75        4.820109
q92        4.515880
q51        4.273222
raceeth    4.103426
q82        4.100723
q60        4.030385
q81        4.025431
q7         3.959653
q54        3.788153
q64        3.787702
q53        3.697364
q17        3.667910
q61        3.638649
q84        3.559189
q10        3.557141
q59        3.547775
q52        3.528288
q12        3.478007
q80        3.446143
q79        3.428434
q2         3.423419
q8         3.411996
q65        3.405296
q29        3.277498
q85        3.266492
q3         3.236468
q58        3.202972
q55        3.198673
q6         3.150844
q1         3.137364
q28        3.130185
q49        3.115456
q9         2.943944
q11        2.846949
q19        2.791596
q87        2.756713
q26        2.717906
q31        2.600180
q4         2.557290
q50        2.425624
q39        1.500961
q46        1.463239
q30        1.371177
q35        1.107665
q38        0.798624
dtype: float32
